In [2]:
import pymysql.cursors
import json
import pandas as pd

In [3]:
host="mysql-rfam-public.ebi.ac.uk"
user="rfamro"
password="none"
port=4497
database="Rfam"

In [4]:
def execute_query(sql):
    connection = pymysql.connect(host=host,
                             port=port,
                             user=user,
                             password=None,
                             database=database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    with connection:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            result = cursor.fetchall()
            print(json.dumps(result,indent=2))
            return result

### 1 How many types of tigers can be found in the taxonomy table of the dataset? What is the "ncbi_id" of the Sumatran Tiger?
#### using the biological name of the tiger: tigris

In [71]:
q1_a="""
SELECT count(species) FROM taxonomy 
WHERE species LIKE '%tigris%';
"""
print("Types of tigers can be found in the taxonomy table ")
res=execute_query(q1_a)

Types of tigers can be found in the taxonomy table 
[
  {
    "count(species)": 29
  }
]


In [72]:
q1_b="""
SELECT ncbi_id FROM taxonomy 
WHERE species LIKE '%sumatran tiger%';
"""
print("ncbi_id of the Sumatran Tiger")
res=execute_query(q1_b)

ncbi_id of the Sumatran Tiger
[
  {
    "ncbi_id": 9695
  }
]


### 2 Find all the columns that can be used to connect the tables in the given database.

In [7]:
#

### 3 Which type of rice has the longest DNA sequence?
#### (using the rfamseq and the taxonomy tables)

In [68]:
q3="""
SELECT max(length),species,tree_display_name,b.ncbi_id FROM rfamseq a 
INNER JOIN (
    SELECT ncbi_id,species,tree_display_name FROM taxonomy 
    WHERE species LIKE '%oryza%sativa%'
) b ON a.ncbi_id = b.ncbi_id
WHERE a.mol_type="genomic DNA"
ORDER BY length DESC
"""
res=execute_query(q3)

[
  {
    "max(length)": 47244934,
    "species": "Oryza sativa (rice)",
    "tree_display_name": "Oryza_sativa_(rice)",
    "ncbi_id": 4530
  }
]


### 4 paginate a list of the family names and their longest DNA sequence lengths 
#### (in descending order of length) where only families that have DNA sequence lengths greater than 1,000,000 are included. Give a query that will return the 9th page when there are 15 results per page. (hint: we need the family accession ID, family name and the maximum length in the results)

In [83]:
q4="""
SELECT final_list.rfam_acc, final_list.length, taxonomy.tree_display_name FROM (
	SELECT family.rfam_acc, length_data.length, length_data.ncbi_id FROM family
	LEFT JOIN
	(
		SELECT full_region.rfam_acc, full_region.rfamseq_acc, rfamseq.length, rfamseq.ncbi_id
		FROM full_region 
		LEFT JOIN rfamseq
		ON full_region.rfamseq_acc = rfamseq.rfamseq_acc LIMIT 0,1000
	) as length_data
    ON family.rfam_acc = length_data.rfam_acc  
    WHERE length_data.length > 1000000 
    ORDER BY length_data.length desc 
    LIMIT 121,135
) as final_list
LEFT JOIN taxonomy on final_list.ncbi_id = taxonomy.ncbi_id;
"""
res=execute_query(q4)

[
  {
    "rfam_acc": "RF00001",
    "length": 50812227,
    "tree_display_name": "Lepisosteus_oculatus_(spotted_gar)"
  },
  {
    "rfam_acc": "RF00001",
    "length": 50348508,
    "tree_display_name": "Lepisosteus_oculatus_(spotted_gar)"
  },
  {
    "rfam_acc": "RF00001",
    "length": 50348508,
    "tree_display_name": "Lepisosteus_oculatus_(spotted_gar)"
  },
  {
    "rfam_acc": "RF00001",
    "length": 50348508,
    "tree_display_name": "Lepisosteus_oculatus_(spotted_gar)"
  },
  {
    "rfam_acc": "RF00001",
    "length": 50348508,
    "tree_display_name": "Lepisosteus_oculatus_(spotted_gar)"
  },
  {
    "rfam_acc": "RF00001",
    "length": 50348508,
    "tree_display_name": "Lepisosteus_oculatus_(spotted_gar)"
  },
  {
    "rfam_acc": "RF00001",
    "length": 50348508,
    "tree_display_name": "Lepisosteus_oculatus_(spotted_gar)"
  },
  {
    "rfam_acc": "RF00001",
    "length": 49578535,
    "tree_display_name": "Callithrix_jacchus_(white-tufted-ear_marmoset)"
  },
  {
    "r

In [82]:
q4="""
SELECT final_list.rfam_acc, final_list.length, taxonomy.tree_display_name FROM (
	SELECT family.rfam_acc, length_data.length, length_data.ncbi_id FROM family
	LEFT JOIN
	(
		SELECT full_region.rfam_acc, full_region.rfamseq_acc, rfamseq.length, rfamseq.ncbi_id
		FROM full_region 
		LEFT JOIN rfamseq
		ON full_region.rfamseq_acc = rfamseq.rfamseq_acc LIMIT 0,100
	) as length_data
    ON family.rfam_acc = length_data.rfam_acc  
    WHERE length_data.length > 1000000 
    GROUP BY family.rfam_acc
    ORDER BY length_data.length desc 
    LIMIT 0,100
) as final_list
LEFT JOIN taxonomy on final_list.ncbi_id = taxonomy.ncbi_id;
"""
res=execute_query(q4)

[
  {
    "rfam_acc": "RF00001",
    "length": 25641242,
    "tree_display_name": "Cavia_porcellus_(Domestic_guinea_pig)"
  }
]
